In [12]:
import json
import requests

In [29]:
coordinates = "55.7263,37.6503,55.7816,37.7864"
text_query = f""" [out: json];
    wr[building]({coordinates})(if:number(t["building:levels"]) >= 1);
    out geom; """

In [ ]:
overpass_url = "https://overpass.openstreetmap.ru/api/interpreter"
response = requests.get(overpass_url, params={'data': text_query})
json_data = response.json()
del json_data["version"]
del json_data["generator"]
del json_data["osm3s"]
with open("output_data.json", "w") as outfile:
    json.dump(json_data, outfile)